In [1]:
import pandas as pd 
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

from keras import models, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding


Using TensorFlow backend.


### Importing Data


In [5]:
#getting data 
df = pd.read_csv('2048_Training_Data_2.csv',index_col=[0])
df_2 = pd.read_csv('2048_Training_Data_3.csv',index_col=[0])
df_3 = pd.read_csv('2048_Training_Data_4.csv',index_col=[0])


# df = pd.DataFrame(df[:200])
print df.shape
print df_2.shape

df = pd.concat([df,df_2,df_3])

print df.shape

(1037, 18)
(1100, 18)
(3262, 18)


### Formatting data into a time series

In [6]:
training_length = 30
num_keys = 4

x_list = []
y_list = []


#one hot encoding
dummy = pd.get_dummies(df['lastMove'])
df = df.merge(dummy,left_index = True, right_index = True)

#dropping unnessary columns
df = df.drop(columns = ['score','lastMove'])


#creating targets and features in a series form
for i in ( np.arange(len(df) - training_length) ):
    x_list.append(df.iloc[i:(i + training_length),0:16])
    y_list.append(df.iloc[i:(i + training_length),16:20])

x_list_2 = []
y_list_2 = []

for i in x_list:
    x = i.values
    x_list_2.append(x)
x_list_2 = np.array(x_list_2)

print type(x_list_2)

# print x_list_2.shape()

for i in y_list:
    y = i.values
    y_list_2.append(y[-1])
y_list_2 = np.array(y_list_2)    

print np.shape(x_list_2)
print np.shape(y_list_2)

<type 'numpy.ndarray'>
(9580, 30, 16)
(9580, 4)


## Setting up Convolutional Model

In [ ]:

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)


network = models.Sequential()
network.add(layers.Dense(160,activation='relu',input_shape=(training_length , 16,)))
# network.add(layers.Dense(16,activation='tanh',input_shape=(16,)))
network.add(layers.Dense(4,activation='softmax'))

network.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    
#Early stopping
patienceCount = 10
   
callbacks = [EarlyStopping(monitor='val_loss', patience=patienceCount),
                     ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', 
                     save_best_only=True)]

# train_labels_cat = to_categorical(train_labels)
# test_labels_cat = to_categorical(test_labels)





In [14]:
history = network.fit(x_list_2,y_list_2,
                            epochs=100,
                            batch_size=120)

acc = history.history['acc'][-patienceCount + 1]

network.save_weights('Conv_Net_1.h5')

print('Accuracy: ', acc)

ValueError: Error when checking target: expected dense_2 to have 3 dimensions, but got array with shape (9580, 4)

In [11]:
from keras.models import Sequential
from keras.layers import Activation, Flatten
from keras.optimizers import RMSprop

# (100, training_length, num_features)
model = Sequential()
model.add(layers.Conv2D(100, training_length, 16, activation='relu',
        input_shape=(None, x_list_2.shape[-1])) )
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))

model.compile( loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'] )

model.summary()

/Users/andreyfilimonov/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(100, (30, 16), activation="relu", input_shape=(None, 16))`
  


ValueError: Input 0 is incompatible with layer conv2d_2: expected ndim=4, found ndim=3

### Setting up LSTM Model

In [ ]:
print type(x_list_2)
print type(y_list_2)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

num_features = 16
num_samples = 9580

model = Sequential()

# Embedding layer
model.add(LSTM((10),batch_input_shape = (100, training_length, num_features), 
              return_sequences = False) )

# model.add(LSTM((10),input_dim = 4) )          

# Compile the model
model.compile( loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'] )
    
model.summary()

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(np.array(x_list), np.array(y_list), test_size=.25, random_state=42)

history = model.fit(x_list,  y_list, 
                 epochs=150)
                 


# print model.evaluate(x_list, y_list)


# print type(sequenceList)

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Flatten
from keras.optimizers import RMSprop

# model = Sequential()
# model.add(Flatten(input_shape=(1,16) ))
# model.add(Dense(nb_actions))
# model.add(Activation('softmax'))
# model.compile( loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'] )
